### python -m pip install pythonnet

In [1]:
from pathlib import Path
from enum import Enum

In [5]:
data_folder = Path('./test_data/')
material_data = 'material_data.json'
column_data = 'column_data.json'
cross_section_data = 'cross_section_data.json'


#start ETABS application.
myETABSObject.ApplicationStart()

#close the program.
ret = myETABSObject.ApplicationExit(False)


In [12]:
from enum import IntEnum
class eReturnCode(IntEnum):
    NotApplicable = -100  
    NotImplemented = -99  
    NoError = 0  
    UnspecifiedError = 1  
    Deprecated = -98  
    TableIsObsolete = -97  
    TableDoesNotExist = -96 


In [11]:
import clr
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

#set the following path to the installed ETABS program directory
clr.AddReference(R'C:\Program Files\Computers and Structures\ETABS 22\ETABSv1.dll')
import ETABSv1 as etabs

#create API helper object
helper = etabs.cHelper(etabs.Helper())

try:
    myETABSObject = etabs.cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
except:
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

#create SapModel object
SapModel = etabs.cSapModel(myETABSObject.SapModel)


display(myETABSObject)
display(SapModel)

SapModel = None
myETABSObject = None

